# Dataset Pre-Processor

## Purpose

This notebook is to prepare raw molecular coordinates for input into the `xyz_training_model` notebook to train the **Equilibrium Molecular Classifier**

<br>

---

In [ ]:
import os
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder

### Raw Data

You will begin with raw csv files containing molecule spatial coordinates in the following format:

```
energy_gap,0.332675
C,2.152789,1.401974,0.597515
C,1.998043,0.223677,-0.359047
O,3.149622,-0.608604,-0.386871
C,0.782749,-0.648626,-0.041220
O,-0.396569,0.166622,-0.190805
C,-1.564498,-0.425885,0.068045
N,-1.683472,-1.634127,0.433307
C,-2.672251,0.496153,-0.122281
N,-3.596210,1.177815,-0.253258
H,3.031185,1.993769,0.327938
H,1.270360,2.046388,0.569160
H,2.277629,1.048408,1.629434
H,1.889483,0.597758,-1.383445
H,3.390302,-0.814685,0.523311
H,0.728477,-1.499685,-0.725138
H,0.822759,-1.037900,0.983223
H,-2.657830,-1.887772,0.578517
```

The below paths are pointing to these files with the energy_gap added via the `energy_calculations.py` script

### Input File Paths
- Please ensure that you have `qm9_data_with_energies` downloaded and saved into the relevant directory that you wish to work from.
- Ensure that both `qm9_data_energy_gap` and`qm9_data_modified_energy_gap` have been included in the `qm9_data_with_energies`, with the path names for `equilibrium_path` and `nonequilibrium_path` below both correctly pointing to these files.

In [ ]:
# --- GOOGLE DRIVE --- #
# If you are running this in Google Colab, using your Google Drive for the dataset, run this cell.
from google.colab import drive

drive.mount('/content/drive')

# Then in the next cell, set the path to your directories in Google Drive.
# e.g.
equilibrium_path = '/content/drive/MyDrive/qm9_data_with_energies/qm9_data_energy_gap'
nonequilirbium_path = '/content/drive/MyDrive/qm9_data_with_energies/qm9_data_modified_energy_gap'

In [ ]:
# --- LOCAL FILES --- #

# Change the file path names such that they point to the correct directories where these files are stored on your local machine
equilibrium_path = "/Users/lyamtalbot/Library/CloudStorage/OneDrive-UniversityofNewEngland/cosc320-project-group-b/qm9_data_with_energies/qm9_data_energy_gap"
nonequilirbium_path = "/Users/lyamtalbot/Library/CloudStorage/OneDrive-UniversityofNewEngland/cosc320-project-group-b/qm9_data_with_energies/qm9_data_modified_energy_gap"

### OneHotEncoder

Using the [scikit-learn OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) to encode our atom types and coordinates into one feature vector.

Neural Networks require numerical inputs so we are creating a way for the model to identify different atoms it would look like `[1, 0, 0, 0, 0, 1.456, 1.432, 0.456]` for a Hydrogen atom.

Here we are just creating the encoder for use later when preparing the data.

In [ ]:
qm9_atoms = ['H', 'C', 'N', 'O', 'F']
feature_dim = len(qm9_atoms) + 3 # feature_dim = number of elements + spatial coordinates (x,y,z) i.e. (3)
encoder = OneHotEncoder(categories=[qm9_atoms], sparse_output=False, handle_unknown='ignore')
encoder.fit(np.array(qm9_atoms).reshape(-1, 1))

### Parsing Comma Seperated Values

This function takes in a file with the structure shown earlier (plus the energy gap) and extracts the energy gap, atom, and coordinates.

In [ ]:
def parse_comma_files(file_path):
  atoms, coords = [], []
  with open(file_path, 'r') as file:
    energy_gap = file.readline().split(',')[1]
    for line in file:
      single_atom = line.strip().split(',')
      if len(single_atom) == 4:
        atoms.append(single_atom[0])
        coords.append([float(val) for val in single_atom[1:]])
  return atoms, np.array(coords), energy_gap

### Adjacency Calculations

- linalg (linear algebra from NumPy)
- Computes pairwise distances between atoms using Euclidean norm

Calculates the distance between atoms and considers them as connected if the distance is between 0 and 1.

In [ ]:
def dense_adj(coords):
    dists = np.linalg.norm(coords[:, None] - coords[None, :], axis=-1)
    adj = ((dists < 1.0) & (dists > 0)).astype(np.float32)
    return adj

Calculates the actual distance between the atoms.

In [ ]:
def distance_mat(coords):
    dists = np.linalg.norm(coords[:, None] - coords[None, :], axis=-1)
    return dists

### Defining the Real Bonds

In [ ]:
# In QM9 molecules, common elements like carbon (C), nitrogen (N), oxygen (O), hydrogen (H), and fluorine (F)
# generally exhibit valencies of 4, 3, 2, 1, and 1, respectively

# We need to create a mask for the above adjacency matrix that when applied to said matrix it will yield a new matrix
# that only contains entries between atoms that are actually bonded.
# Atoms could theoretically be bonded if their euclidean distance <= the sum of the two atoms covalent radii +/- [0.1,0.4] angstroms
# We also need to check the valence of the atoms to ensure that atoms have only the number of bonds they can support
# If two atoms are the same distance away we'll probably need to go by some other rule like charge?

# Can probably use the valency count as a loop counter
# Determine what the element is and therefore how many bonds it can support
# We then loop that many times to see what else it's bonded to.
# Sort all other atoms by their distance to this one
# Bond with the one with the shortest distance
# Remove that from the list
# Bond with the next closest
# One thing to consider though is while atom_2 might be close enough to atom_1 to form a bond based on
# distance, there might be some atom_3 that is closer to atom_2 than atom_1 and in the case where atom_2 is something like hydrogen
# with a valency of 1, it would likely bond to the closer atom.
# so I'll probably need to run these checks in parallel for both atoms to ensure the right bond is chosen
# both atoms will need to "agree" that they should be bonded in order for the bond to be established
# one we've established that a bond exists between these two atoms we can make bond_mask[i][j] == bond_mask[j][i] == 1, all atom pairs that
# do not have bonds will == 0 instead.
# when this mask is applied to the fully connected weighted adjacency matrix it will 0 out all the distances for
# atom pairs that are not chemically bonded. So we would have a matrix that represents the bonds and their lengths.


# dense_adj is a matrix representing all the connections from every atom to every other atom
# atoms is a list of the atoms(in order)
# valancies could be a global which represents the valency of this atom within this system
# covalent radii could also be a global, represents the distance at which an atom forms a covalent bond with itself
# if two elements are close enough such that their distance is less than or equal to the sum of their covalent radii we can
# infer that they are "probably" bonded
# I already have the list of every atom's distance to every other atom.
# I just need to track how many bonds are actually free for each atom.
# I could create another matrix that would track how many bonds each atom has remaining
# Oh hang on I could maybe use the diagonal of the real bonds matrix to store the valency of the atoms which would allow me to the entries on BOTH atoms
# This way I don't end up making too many bonds by mistake

# [[2,0,0,0,0,0], The valency matrix may look something like this.
#  [0,3,0,0,0,0], valency_matrix[i][i] represents the number of remaining bonds for this atom
#  [0,0,1,0,0,0], When we make a bond with another atom, provided we know it's index
#  [0,0,0,4,0,0], we can decrement valency_matrix[other_atom][other_atom].
#  [0,0,0,0,2,0], hopefully then we can avoid over bonding any atoms.
#  [0,0,0,0,0,1]]

# In QM9 molecules, common elements like carbon (C), nitrogen (N), oxygen (O), hydrogen (H), and fluorine (F)
# generally exhibit valencies of 4, 3, 2, 1, and 1, respectively

# qm9_atoms = ['H', 'C', 'N', 'O', 'F']
valencies = {'H': 1, 'C': 4, 'N': 3, 'O': 2, 'F': 1}
covalent_radii = {'H': 0.31, 'C': 0.76, 'N': 0.71, 'O':0.66, 'F' : 0.57} #covalent radii in angstroms

def define_real_bonds(dense_adj, atoms):
    #each row in this matrix represents 1 atom and it's bonds to every other atom in the molecule
    #they should be in the same order as the atoms so row 1 -> atom 1, etc.
    real_bonds = np.zeros((len(dense_adj), len(dense_adj)))
    for i in range(len(atoms)):
        real_bonds[i][i] = valencies[atoms[i]]
        #now the diagonal of the matrix should contain the valency of this atom
        #we can decrement that every time we make a bond to keep track of the number of bonds for both atoms
        #we may still end up in a position where we use a make a bond to an atom because
        #it is our closest neighbour however it may have another neighbour that is closer that it should be bonded to instead
        #additionally we want make sure in the case of hyrdogen and fluorine that we're only bonding to atoms that have a valency >= 2
        #this should prevent them from bonding to each other, because if that were to happen they wouldn't be connected to the rest of the molecule anymore.

    for i in range(len(dense_adj)):
        candidate_bonds = list(enumerate(dense_adj[i])) ## all the bonds with this atom and their index
        candidate_bonds.pop(i) ## want to remove the column that corresponds to this atom, it doesn't have a bond to itself.
        candidate_bonds.sort(key = lambda x:x[1]) ## sort the candidates on distance.
        # valency = real_bonds[i][i]
        self_conv_radius = covalent_radii[atoms[i]] ##get the covalent radius for this element
        for bond in candidate_bonds:
            ## can play around with what counts as a bond length, what shift we're happy to accept
            ## I'd had this at 0.3 angstroms but that's way too high
            ## changing to 0.1 increased the accuracy quite a bit
            ## 0.05 also seems to work well.
            if bond[1] <= self_conv_radius + covalent_radii[atoms[bond[0]]] + 0.05 and real_bonds[i][i] > 0 and real_bonds[bond[0]][bond[0]] > 0:
                real_bonds[i][bond[0]] = 1
                real_bonds[bond[0]][i] = 1
                real_bonds[i][i] -= 1
                real_bonds[bond[0]][bond[0]] -= 1
            else:
                continue
    return real_bonds

        # for j in range(valency):
        #     for k in range(len(candidate_bonds)):
        #         if candidate_bonds[k][1] <= (covalent_radii[atom] + covalent_radii[])

# my notes are a mess
# if this doesn't work we might want to order all the pairwise distances in the matrix from smallest to largest
# then go through in order and check if those


### Adding Padding

Pads an array to a new shew shape so we can get all molecules to the same shape.

In [ ]:
def pad_array(arr, new_shape):
    p_val = 0.0
    padded = np.full(new_shape, p_val, dtype=arr.dtype)
    padded[:arr.shape[0], :arr.shape[1]] = arr
    return padded

### Encode the Atoms

- Uses our encoder created at the start to encode the atom characters to ints.

In [ ]:
def encode_labels(atoms, coords):
    # Reshaping the atoms into one per row for the encoder
    shaped_atoms = np.array(atoms).reshape(-1, 1)
    # Use our encoder on the atoms
    encoded_atoms = encoder.transform(shaped_atoms).astype(np.float32)
    # Add the coords to the encoded atoms
    return np.concatenate([encoded_atoms, coords], axis=-1)

### Create a Mask

- Generates a mask where 0 is no molecule and 1 is a real molecule.
- This allows us to pad the smaller molecules to the size of the largest and the model can know which nodes are real.

In [ ]:
def create_mask(encoded_atoms, max_nodes):
    # Create a list of zeros with the size of max_nodes
    mask = np.zeros((max_nodes,), dtype=np.float32)
    # Set the first num_real elements to 1.0
    mask[:encoded_atoms.shape[0]] = 1.0
    return mask

### Prepare Data

This function extracts the data from the input folders and forms the various features and labels we need.

We return:
- `molecules`: Array of the individual atoms
- `adj_arr`: Adjacency list of real bonds
- `mask_arr`: Masking array with 0 for no bond or 1 for a bond
- `labels`: Original labels for each molecule (1 for equilibrium and 0 for nonequilibrium)
- `max_nodes`: Maximum number of nodes found in a single molecule
- `energy_gaps`: The pre-calculated energy gap for each file

In [ ]:
def prepare_data(equ_dir, non_equ_dir):
    node_list, adj_list, mask_list, label_list, energy_gaps = [], [], [], [], []
    all_files = [] # Temp list to collect all files and labels

    # Iterate over both equ_dir and non_equ_dir folders
    # Collect the file paths and labels together
    for label, folder in [(1, equ_dir), (0, non_equ_dir)]:
        for file_name in os.listdir(folder):
            file_path = os.path.join(folder, file_name)
            all_files.append((file_path, label))

    # Sort to ensure consistent order
    all_files.sort(key=lambda x: x[0])

    max_nodes = 0 # Max nodes found in any molecule
    temp_graphs = [] # Temp storage for the next loop

    # Go through each file-label pair and build a temp dataset
    for file_path, label in all_files:
        atoms, coords, energy_gap = parse_comma_files(file_path)
        temp_graphs.append((atoms, coords, energy_gap, label))
        max_nodes = max(max_nodes, len(atoms))

    # Process each molecule
    for atoms, coords, energy_gap, label in temp_graphs:
        # Encoded the atoms and coords
        encoded_atoms = encode_labels(atoms, coords)

        # --- Components to Return (plus max_nodes) --- This order is important
        mask_list.append(create_mask(encoded_atoms, max_nodes))
        node_list.append(pad_array(encoded_atoms, (max_nodes, encoded_atoms.shape[1])))
        adj_list.append(pad_array(define_real_bonds(distance_mat(coords), atoms), (max_nodes, max_nodes)))
        label_list.append(np.array([label], dtype=np.float32))
        energy_gaps.append([energy_gap])

    return (
        np.array(node_list),
        np.array(adj_list),
        np.array(mask_list),
        np.array(label_list),
        max_nodes,
        np.array(energy_gaps, dtype=np.float32),
    )

### Create the TensorFlow Dataset

This is the required dataset structure for the model.

In [ ]:
def create_tf_ds(molecules, adj_arr, mask_arr, labels, energy_gap, batch_size=32, shuffle=True, drop_remainder=False):
    # Reshape energy_gap to ensure it has shape [?, 1]
    energy_gap = energy_gap.reshape(-1, 1) # Convert to 2D array

    ds = tf.data.Dataset.from_tensor_slices(((molecules, adj_arr, mask_arr, energy_gap), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(molecules))
    # ds = ds.batch(batch_size, drop_remainder=drop_remainder)
    return ds


### Load and process our data

- `molecules`: The actual atom types (encoded as integers).
- `adj_arr`: Identifying which atoms are actually bonded.
- `mask_arr`: arrays of 0's and 1's to indicate to the model whether that node is a real atom or padding.
- `labels`: Each molecule gets a 0 or 1 label as to whether it is in equilibrium or not for training.
- `max_nodes`: The maximum number of atoms found in a single molecule.
- `energy_gaps`: A list of the pre-calculated energy gaps for each molecule.


In [ ]:
molecules, adj_arr, mask_arr, labels, max_nodes, energy_gaps = prepare_data(equilibrium_path, nonequilirbium_path)
print("Loaded", molecules.shape[0], "graphs, each padded to", max_nodes, "nodes")

Loaded 20000 graphs, each padded to 26 nodes


### Specify Output Directories

- `train_dir`: where to save your training data.
- `test_dir`: where to save your testing data.

In [ ]:
train_dir = "/Users/lyamtalbot/Library/CloudStorage/OneDrive-UniversityofNewEngland/cosc320-project-group-b/qm9_tensorflow_datasets/qm9_03/qm9_03_train"
test_dir = "/Users/lyamtalbot/Library/CloudStorage/OneDrive-UniversityofNewEngland/cosc320-project-group-b/qm9_tensorflow_datasets/qm9_03/qm9_03_test"

### Create and Save the Datasets

Creates the TensorFlow dataset, splits into train and test and saves to the path provided.

In [ ]:
dataset = create_tf_ds(molecules, adj_arr, mask_arr, labels, energy_gaps, shuffle=True)

qm9_train, qm9_test = tf.keras.utils.split_dataset(dataset, left_size=0.8, shuffle=False)

qm9_train.save(path=train_dir)
qm9_test.save(path=test_dir)